In [1]:
import asyncio
import awswrangler as wr
from datetime import datetime, timedelta
from IPython.display import display
from metaapi_cloud_sdk import MetaApi
import os
import pandas as pd
import pytz
import sys
from time import sleep

In [2]:
repo_name = "mercury-pipeline-metaapi-sam"
sys.path.append(os.getcwd().split(repo_name)[0] + repo_name + "/lambda") 

In [3]:
from src.config import *
from src.io import *
from src.utils import *
from src.access import get_demo_account

In [4]:
from src.ticks.clean.get import get_clean_ticks
from src.candles.from_ticks.get import get_candles_from_ticks
from src.candles.transform.save import ffill

In [5]:
async def yield_raw_ticks_between_datetimes(
    account: object,
    symbol: str,
    datetimeutc_from: datetime,
    datetimeutc_to: datetime,
    pause: int = 5
):
    offset = 0
    while True:
        l_ticks_i = await account.get_historical_ticks(
            symbol,
            start_time=datetimeutc_from,
            limit=1000,
            offset=offset,
        )
        if len(l_ticks_i) == 0:
            break
        yield offset, l_ticks_i
        await asyncio.sleep(pause)
        datetimeutc_last = l_ticks_i[-1]["time"]
        offset += len(l_ticks_i)
        if datetimeutc_last >= datetimeutc_to:
            break

In [6]:
symbol = "GBPUSD"
frequency = "15T"

In [7]:
datetimestr_from = (
    pytz
    .timezone("Europe/London")
    .localize(datetime.now())
    .astimezone(pytz.timezone("UTC"))
    .strftime("%Y%m%d000000000000")
)

datetimestr_to = (
    pytz
    .timezone("Europe/London")
    .localize(datetime.now() + timedelta(days=1))
    .astimezone(pytz.timezone("UTC"))
    .strftime("%Y%m%d%H%M%S%f")
)
datetimestr_from, datetimestr_to

('20220708000000000000', '20220709155124024878')

In [8]:
# Get demo account
account = await get_demo_account()

# Convert datetimes from string to datetime
datetimeutc_from = pytz.timezone("UTC").localize(datetime.strptime(datetimestr_from, "%Y%m%d%H%M%S%f"))
datetimeutc_to = pytz.timezone("UTC").localize(datetime.strptime(datetimestr_to, "%Y%m%d%H%M%S%f"))

In [9]:
generator_async = yield_raw_ticks_between_datetimes(
    account=account,
    symbol=symbol,
    datetimeutc_from=datetimeutc_from,
    datetimeutc_to=datetimeutc_to,
)

In [10]:
i = 0
df_ticks_clean_prev = pd.DataFrame()
async for offset, ticks_raw in generator_async:
    
    print(offset)
    
    if ticks_raw == []:
        sleep(10)
        continue

    df_ticks_clean = get_clean_ticks(ticks_raw)
    df_ticks_clean = pd.concat([
        df_ticks_clean_prev,
        df_ticks_clean
    ])
    
    df_candles_from_ticks = get_candles_from_ticks(
        df_ticks_clean,
        frequency
    )

    df_candles_ffill = ffill(df_candles_from_ticks)
    
    s_candle_last = df_candles_ffill.iloc[-1]
    df_candles_out = df_candles_ffill.iloc[:-1].copy() # could be empty so beware
    
    mask_last = df_ticks_clean["timestamp_utc"] >= s_candle_last["timestamp_utc.open"]
    df_ticks_clean_prev = df_ticks_clean[mask_last].copy()
    
    display(df_candles_out)

    i += 1
    if i >= 20:
        break
    sleep(1)

0


,symbol,frequency,year,month,date,hour,timestamp_utc,timestamp_uk,timestamp_broker,bid,ask,last,volume
0,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:00:00.083,2022-07-08 01:00:00.083,2022-07-08 03:00:00.083,NaN,NaN,1.20271,150000.0
1,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:00:00.083,2022-07-08 01:00:00.083,2022-07-08 03:00:00.083,1.20271,1.20321,NaN,NaN
2,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:00:00.131,2022-07-08 01:00:00.131,2022-07-08 03:00:00.131,NaN,NaN,1.20291,150000.0
3,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:00:00.132,2022-07-08 01:00:00.132,2022-07-08 03:00:00.132,1.20291,1.20303,NaN,NaN
4,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:00:00.234,2022-07-08 01:00:00.234,2022-07-08 03:00:00.234,NaN,NaN,1.20290,150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.135,2022-07-08 01:01:58.135,2022-07-08 03:01:58.135,NaN,NaN,1.20321,150000.0
996,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.236,2022-07-08 01:01:58.236,2022-07-08 03:01:58.236,NaN,NaN,1.20321,150000.0
997,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.337,2022-07-08 01:01:58.337,2022-07-08 03:01:58.337,NaN,NaN,1.20321,150000.0
998,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.438,2022-07-08 01:01:58.438,2022-07-08 03:01:58.438,NaN,NaN,1.20321,150000.0


1000


,symbol,frequency,year,month,date,hour,timestamp_utc,timestamp_uk,timestamp_broker,bid,ask,last,volume
0,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:00:00.083,2022-07-08 01:00:00.083,2022-07-08 03:00:00.083,NaN,NaN,1.20271,150000.0
1,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:00:00.083,2022-07-08 01:00:00.083,2022-07-08 03:00:00.083,1.20271,1.20321,NaN,NaN
2,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:00:00.131,2022-07-08 01:00:00.131,2022-07-08 03:00:00.131,NaN,NaN,1.20291,150000.0
3,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:00:00.132,2022-07-08 01:00:00.132,2022-07-08 03:00:00.132,1.20291,1.20303,NaN,NaN
4,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:00:00.234,2022-07-08 01:00:00.234,2022-07-08 03:00:00.234,NaN,NaN,1.20290,150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.041,2022-07-08 01:04:07.041,2022-07-08 03:04:07.041,NaN,NaN,1.20332,150000.0
996,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.142,2022-07-08 01:04:07.142,2022-07-08 03:04:07.142,NaN,NaN,1.20332,150000.0
997,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.420,2022-07-08 01:04:07.420,2022-07-08 03:04:07.420,NaN,NaN,1.20332,150000.0
998,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.519,2022-07-08 01:04:07.519,2022-07-08 03:04:07.519,NaN,NaN,1.20332,150000.0


2000


...

,symbol,frequency,year,month,date,hour,timestamp_utc,timestamp_uk,timestamp_broker,bid,ask,last,volume
811,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:00.171,2022-07-08 01:45:00.171,2022-07-08 03:45:00.171,NaN,NaN,1.20317,150000.0
812,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:00.172,2022-07-08 01:45:00.172,2022-07-08 03:45:00.172,1.20317,NaN,NaN,NaN
813,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:00.275,2022-07-08 01:45:00.275,2022-07-08 03:45:00.275,NaN,NaN,1.20317,150000.0
814,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:00.376,2022-07-08 01:45:00.376,2022-07-08 03:45:00.376,NaN,NaN,1.20317,150000.0
815,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:00.477,2022-07-08 01:45:00.477,2022-07-08 03:45:00.477,NaN,NaN,1.20317,150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:26.569,2022-07-08 01:45:26.569,2022-07-08 03:45:26.569,NaN,NaN,1.20323,150000.0
996,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:27.026,2022-07-08 01:45:27.026,2022-07-08 03:45:27.026,NaN,NaN,1.20323,150000.0
997,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:27.134,2022-07-08 01:45:27.134,2022-07-08 03:45:27.134,NaN,NaN,1.20323,150000.0
998,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:28.632,2022-07-08 01:45:28.632,2022-07-08 03:45:28.632,NaN,NaN,1.20323,150000.0


18000


,symbol,frequency,year,month,date,hour,timestamp_utc,timestamp_uk,timestamp_broker,bid,ask,last,volume
811,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:00.171,2022-07-08 01:45:00.171,2022-07-08 03:45:00.171,NaN,NaN,1.20317,150000.0
812,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:00.172,2022-07-08 01:45:00.172,2022-07-08 03:45:00.172,1.20317,NaN,NaN,NaN
813,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:00.275,2022-07-08 01:45:00.275,2022-07-08 03:45:00.275,NaN,NaN,1.20317,150000.0
814,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:00.376,2022-07-08 01:45:00.376,2022-07-08 03:45:00.376,NaN,NaN,1.20317,150000.0
815,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:00.477,2022-07-08 01:45:00.477,2022-07-08 03:45:00.477,NaN,NaN,1.20317,150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:47:31.163,2022-07-08 01:47:31.163,2022-07-08 03:47:31.163,NaN,NaN,1.20410,150000.0
996,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:47:31.264,2022-07-08 01:47:31.264,2022-07-08 03:47:31.264,NaN,NaN,1.20410,150000.0
997,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:47:31.365,2022-07-08 01:47:31.365,2022-07-08 03:47:31.365,NaN,NaN,1.20410,150000.0
998,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:47:31.489,2022-07-08 01:47:31.489,2022-07-08 03:47:31.489,NaN,NaN,1.20410,150000.0


19000


,symbol,frequency,year,month,date,hour,timestamp_utc,timestamp_uk,timestamp_broker,bid,ask,last,volume
811,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:00.171,2022-07-08 01:45:00.171,2022-07-08 03:45:00.171,NaN,NaN,1.20317,150000.0
812,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:00.172,2022-07-08 01:45:00.172,2022-07-08 03:45:00.172,1.20317,NaN,NaN,NaN
813,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:00.275,2022-07-08 01:45:00.275,2022-07-08 03:45:00.275,NaN,NaN,1.20317,150000.0
814,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:00.376,2022-07-08 01:45:00.376,2022-07-08 03:45:00.376,NaN,NaN,1.20317,150000.0
815,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:45:00.477,2022-07-08 01:45:00.477,2022-07-08 03:45:00.477,NaN,NaN,1.20317,150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:50:11.560,2022-07-08 01:50:11.560,2022-07-08 03:50:11.560,NaN,NaN,1.20378,150000.0
996,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:50:11.661,2022-07-08 01:50:11.661,2022-07-08 03:50:11.661,NaN,NaN,1.20378,150000.0
997,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:50:11.761,2022-07-08 01:50:11.761,2022-07-08 03:50:11.761,NaN,NaN,1.20378,150000.0
998,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:50:12.505,2022-07-08 01:50:12.505,2022-07-08 03:50:12.505,NaN,NaN,1.20378,150000.0


In [11]:
assert False

AssertionError: 

In [ ]:
offset, ticks_raw = await generator_async.__anext__()

In [ ]:
ticks_raw[0], ticks_raw[-1]

In [ ]:
df_ticks_clean = get_clean_ticks(ticks_raw)
df_ticks_clean

In [ ]:
df_candles_from_ticks = get_candles_from_ticks(
    df_ticks_clean,
    frequency
)
df_candles_from_ticks

In [ ]:
df_candles_ffill = ffill(df_candles_from_ticks)
df_candles_ffill

In [ ]:
s_candle_last = df_candles_ffill.iloc[-1]
s_candle_last

In [ ]:
df_candles_out = df_candles_ffill.iloc[:-1].copy()
df_candles_out

In [ ]:
df_ticks_clean_last = df_ticks_clean[df_ticks_clean["timestamp_utc"] >= s_candle_last["timestamp_utc.open"]].copy()
df_ticks_clean_last

In [ ]:
offset, ticks_raw = await generator_async.__anext__()

In [ ]:
df_ticks_clean = get_clean_ticks(ticks_raw)
df_ticks_clean

In [ ]:
df_ticks_clean = pd.concat([
    df_ticks_clean_last,
    df_ticks_clean
])
df_ticks_clean

In [ ]:
df_candles_from_ticks = get_candles_from_ticks(
    df_ticks_clean,
    frequency
)
df_candles_from_ticks

In [ ]:
df_candles_ffill = ffill(df_candles_from_ticks)
df_candles_ffill

In [ ]:
s_candle_last = df_candles_ffill.iloc[-1]
s_candle_last

In [ ]:
df_candles_out = df_candles_ffill.iloc[:-1].copy()
df_candles_out

In [ ]:
df_ticks_clean_last = df_ticks_clean[df_ticks_clean["timestamp_utc"] >= s_candle_last["timestamp_utc.open"]].copy()
df_ticks_clean_last